In [ ]:
import spacy
from spacy import displacy 
from spacy.symbols import nsubj, VERB, obj
#!python3 -m spacy download pt_core_news_sm

In [ ]:
class AugmenterParser():
    
    def __init__(self, frase):
        nlp = spacy.load("pt_core_news_sm")
        self.doc = nlp(frase)
        
    def find_root(self):
        root = ''
        for token in self.doc:
            if token.dep_ == 'ROOT': 
                root = token.text
                root_position = token.i
        return root, root_position
    
    def get_children(self):
        children = []

        for token in self.doc:
            if token.dep == nsubj and token.head.pos == VERB:
                children.append((token.text, token.i))

            if token.dep == obj and token.head.pos == VERB:
                children.append((token.text, token.i))

        return children    
    
    def get_itens(self):
        itens = []
        root, root_i = self.find_root()
        children = self.get_children()

        for child in children:
            subj, subj_i = child
            center = []
            for descendant in self.doc[subj_i].subtree:   
                if descendant.head.i != root_i and descendant.head.i not in center:
                    itens.append((f'{descendant.text} {descendant.head.text}', descendant.head.i, descendant.i))
                    center.append(descendant.head.i)
        return itens
    
    def crop(self):
        cropped_sentences = []
        root, root_i = self.find_root()
        itens = self.get_itens()

        if len(self.doc) > 1:
            for i in range(len(itens)):
                sub_frase, head_i, sub_i = itens[i]
                if sub_i == root_i:
                    continue
                elif sub_i < root_i:
                    new = f'{sub_frase} {self.doc[root_i]}'
                elif sub_i > root_i:
                    new = f'{self.doc[root_i]} {sub_frase}'

                cropped_sentences.append(new)

        return cropped_sentences
    
    def show_tree(self):
        return displacy.render(self.doc)

In [ ]:
ag = AugmenterParser('seu pai escreveu uma carta para ela')
ag.crop()

['seu pai escreveu', 'escreveu uma carta', 'escreveu para ela']

In [ ]:
ag.show_tree()